In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve
import caas_jupyter_tools as jt

In [3]:
df=pd.read_excel("/kaggle/input/logistic-regression-hw/Employee.xlsx")
df

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,Bachelors,2017,Bangalore,3,34,Male,No,0,0
1,Bachelors,2013,Pune,1,28,Female,No,3,1
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0
3,Masters,2016,Bangalore,3,27,Male,No,5,1
4,Masters,2017,Pune,3,24,Male,Yes,2,1
...,...,...,...,...,...,...,...,...,...
4648,Bachelors,2013,Bangalore,3,26,Female,No,4,0
4649,Masters,2013,Pune,2,37,Male,No,2,1
4650,Masters,2018,New Delhi,3,27,Male,No,5,1
4651,Bachelors,2012,Bangalore,3,30,Male,Yes,2,0


In [8]:
target_candidates = [c for c in df.columns if c.lower() in ("left","attrition","resigned","is_left")]
print("\nDetected target-like columns:", target_candidates)


Detected target-like columns: []


In [24]:
if 'left' in df.columns:
    target = 'left'
elif target_candidates:
    target = target_candidates[0]
else:
    # try common 'Attrition' variants
    if 'Attrition' in df.columns:
        target = 'Attrition'
    else:
        # fallback: if there's a column with values 0/1 we might choose it; else raise
        binary_cols = [c for c in df.columns if df[c].nunique()<=3 and set(df[c].dropna().unique()).issubset({0,1,2})]
        if binary_cols:
            target = binary_cols[0]
        else:
            raise ValueError("Couldn't detect a target column automatically. Expected a 'left' or similar column.")

print("\nUsing target column:", target)


Using target column: LeaveOrNot


In [25]:
print("\nTarget value counts:\n", df[target].value_counts())


Target value counts:
 LeaveOrNot
0    3053
1    1600
Name: count, dtype: int64


In [26]:
numeric = df.select_dtypes(include=[np.number]).columns.tolist()
if target in numeric:
    numeric_no_target = [c for c in numeric if c!=target]
else:
    numeric_no_target = numeric
corrs = df[numeric_no_target + [target]].corr()[target].drop(target).sort_values(key=lambda x: x.abs(), ascending=False)
print("\nNumeric features correlation with target (sorted by absolute value):\n", corrs)


Numeric features correlation with target (sorted by absolute value):
 PaymentTier                 -0.197638
JoiningYear                  0.181705
Age                         -0.051126
ExperienceInCurrentDomain   -0.030504
Name: LeaveOrNot, dtype: float64


In [ ]:
corr_df = corrs.reset_index().rename(columns={'index':'feature', target:'corr_with_target'})
jt.display_dataframe_to_user("Correlation with target", corr_df)


In [ ]:
salary_cols = [c for c in df.columns if 'salary' in c.lower()]
dept_cols = [c for c in df.columns if 'dept' in c.lower() or 'department' in c.lower()]

salary_col = salary_cols[0] if salary_cols else None
dept_col = dept_cols[0] if dept_cols else None

print("\nDetected salary column:", salary_col)
print("Detected department column:", dept_col)

In [ ]:
if salary_col:
    salary_group = df.groupby([salary_col, target]).size().unstack(fill_value=0)
    salary_group['total'] = salary_group.sum(axis=1)
    salary_group['prop_left'] = salary_group.get(1, 0) / salary_group['total'] if 1 in salary_group.columns else 0
    display_df = salary_group.copy()
    jt.display_dataframe_to_user("Salary vs retention counts & proportions", display_df.reset_index())
    # Plot bar chart: counts of left vs stayed per salary
    ax = salary_group.drop(columns=['total','prop_left'], errors='ignore').plot(kind='bar', figsize=(7,4))
    ax.set_title("Counts by Salary and Retention (target={})".format(target))
    ax.set_xlabel("Salary")
    ax.set_ylabel("Count")
    plt.tight_layout()
    plt.show()

In [ ]:
ax = salary_group['prop_left'].plot(kind='bar', figsize=(6,3))
    ax.set_title("Proportion left by Salary")
    ax.set_ylabel("Proportion left")
    plt.tight_layout()
    plt.show()
else:
    print("No salary-like column detected; skipping salary plots.")

if dept_col:
    dept_group = df.groupby([dept_col, target]).size().unstack(fill_value=0)
    dept_group['total'] = dept_group.sum(axis=1)
    dept_group['prop_left'] = dept_group.get(1, 0) / dept_group['total'] if 1 in dept_group.columns else 0
    jt.display_dataframe_to_user("Department vs retention counts & proportions", dept_group.reset_index())
    ax = dept_group.drop(columns=['total','prop_left'], errors='ignore').plot(kind='bar', figsize=(10,4))
    ax.set_title("Counts by Department and Retention (target={})".format(target))
    ax.set_xlabel("Department")
    ax.set_ylabel("Count")
    plt.tight_layout()
    plt.show()

    ax = dept_group['prop_left'].plot(kind='bar', figsize=(10,3))
    ax.set_title("Proportion left by Department")
    ax.set_ylabel("Proportion left")
    plt.tight_layout()
    plt.show()
else:
    print("No department-like column detected; skipping department plots.")

In [ ]:
abs_corr = corrs.abs()
selected_numeric = abs_corr[abs_corr >= 0.10].index.tolist()
if len(selected_numeric) == 0:
    selected_numeric = abs_corr.index.tolist()[:5]
print("\nSelected numeric features for modelling:", selected_numeric)

selected_features = selected_numeric.copy()
if salary_col:
    selected_features.append(salary_col)
if dept_col:
    selected_features.append(dept_col)

print("\nFinal selected features:", selected_features)

In [ ]:
X = df[selected_features].copy()
y = df[target].copy()

In [ ]:
numeric_features = [c for c in selected_numeric if c in X.columns]
cat_features = [c for c in X.columns if c not in numeric_features]

In [ ]:
salary_enc = None
if salary_col and salary_col in cat_features:
    # try to detect order
    uniq = [str(x).lower() for x in sorted(df[salary_col].dropna().unique(), key=str)]
    # common mapping
    order = None
    if set(['low','medium','high']).issubset(set(uniq)):
        order = ['low','medium','high']
    if order:
        cat_transformer = ColumnTransformer(transformers=[
            ('salary_ord', OrdinalEncoder(categories=[order]), [salary_col]),
            ('dept_ohe', OneHotEncoder(handle_unknown='ignore', sparse=False), [dept_col]) if dept_col and dept_col in cat_features else ('drop', 'drop', [])
        ], remainder='drop')

In [ ]:
      preprocessor = ColumnTransformer(transformers=[
            ('num', StandardScaler(), numeric_features),
            ('cat', OneHotEncoder(handle_unknown='ignore', sparse=False), cat_features)
        ], remainder='drop')
    else:
        preprocessor = ColumnTransformer(transformers=[
            ('num', StandardScaler(), numeric_features),
            ('cat', OneHotEncoder(handle_unknown='ignore', sparse=False), cat_features)
        ], remainder='drop')
else:
    preprocessor = ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse=False), cat_features)
    ], remainder='drop')

In [ ]:
pipe = Pipeline([
    ('pre', preprocessor),
    ('clf', LogisticRegression(max_iter=1000))
])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)


In [ ]:
pipe.fit(X_train, y_train)

In [ ]:
y_pred = pipe.predict(X_test)
y_proba = pipe.predict_proba(X_test)[:,1] if hasattr(pipe.named_steps['clf'], "predict_proba") else None

acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
clf_report = classification_report(y_test, y_pred, digits=4)
roc_auc = roc_auc_score(y_test, y_proba) if y_proba is not None else None

print("\nModel performance on test set:")
print("Accuracy:", acc)
if roc_auc is not None:
    print("ROC AUC:", roc_auc)
print("\nConfusion Matrix:\n", cm)
print("\nClassification Report:\n", clf_report)

In [ ]:
feature_names = []
pre = pipe.named_steps['pre']
# Numeric names
if numeric_features:
    feature_names += numeric_features
# Categorical names from OneHotEncoder if used
if cat_features:
    # find transformer
    # we used ColumnTransformer with a 'cat' transformer named 'cat'
    for name, trans, cols in pre.transformers_:
        if name == 'cat':
            ohe = trans
            # handle when sparse=False else get_feature_names_out
            try:
                cat_names = ohe.get_feature_names_out(cols).tolist()
            except Exception:
                cat_names = []
            feature_names += cat_names
            break

coef = pipe.named_steps['clf'].coef_[0]
coef_df = pd.DataFrame({'feature': feature_names, 'coefficient': coef}).sort_values(by='coefficient', key=lambda x: x.abs(), ascending=False)
jt.display_dataframe_to_user("Model coefficients (approx)", coef_df.head(40))

In [ ]:
if y_proba is not None:
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    plt.figure(figsize=(6,4))
    plt.plot(fpr, tpr)
    plt.plot([0,1],[0,1],'--')
    plt.title("ROC Curve (AUC = {:.4f})".format(roc_auc))
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.tight_layout()
    plt.show()

In [ ]:
print("\nSUMMARY:")
print("- Detected target column: {}".format(target))
print("- Numeric features with strongest correlation to the target (absolute value, top listed above).")
print("- Salary and Department (if present) plotted above showing counts and proportions of employees who left.")
print("- Final model used features: {}".format(selected_features))
print("- Test accuracy: {:.4f}".format(acc))
if roc_auc is not None:
    print("- Test ROC AUC: {:.4f}".format(roc_auc))
print("\nFiles created: None. All outputs displayed inline.")